## Cutting flex disc with changing speeds 
The aim of the example is the demonstration of time varying input variables. This is done with an example of a cutting flex disc on an angle grinder with changing speeds. The real world beheaviour is thereby very roughly estimated.

In [1]:
from numpy import set_printoptions, linspace
from math import tau
from thermca import *

Modeling a flex disc with a homogenised material mix from existing database materials.

In [2]:
with Asm() as disc_asm:
    cyl = Cyl(
        inner_rad=.011,
        outer_rad=.075,
        lgth=.0015,
        rad_div=16,
    )
    Surf(
        name='circ_faces',
        faces=[cyl.face.base, cyl.face.end],
    )
    Surf(
        name='outer',
        faces=[cyl.face.outer],
    )
    Surf(
        name='inner',
        faces=[cyl.face.inner],
    )

with Model() as model:
    reinforced_abrasive = Solid.mix(
        matls=(solids.alumina, solids.glass),
        shares=(.9, .1)
    )
    disc = LPPart(
        asm=disc_asm,
        matl=reinforced_abrasive,
        init_temp=20.,
        name='flex_disc',
    )

The measured ambient temperature changes significantly. Hence it is used as a time changing simulation boundary condition. Time changing model inputs are considered with `Input` elements. This elements take a two dimensional sequence in a regular array shape as argument. The first column contains the time steps and the second column the corresponding values.

In [3]:
with model:
    # Time steps and corresponding temperatures
    env_temp = Input(
        [[0, 19],   
         [20, 23],
         [40, 27]],  
        name='environment_temperatures'
    )

Boundary temperatures can be modeled as `BoundNode` point nodes. The `temp` argument can take functions to define time dependent temperatures. The temperature function itself does not take aruments and returns temperatures over simulation time. This kind of function delivers the `Input` model element itself with the `get_value` method.

In [4]:
with model:    
    environment = BoundNode(
        temp=env_temp.get_value,  # Given as function
        name='environment',
    )

At first a small cutting force is induced. After 20 seconds the force gets more powerful. This is results in a lower rotation speed. The time varying speed influences the convective heat transfer from the disc to the surrounding air. This heat transfer is modeled by a speed dependend heat transfer coefficient. For this purpose functions can be find in the `forced_conv` library. The functions return functions compatible with `FilmLink` elements (required arguments: surface temperature, surrounding fluid temperature and fluid). The keyword arguments of the library functions can be used to modify input parameters of the returned functions. The parameters may be time dependent like the rotation frequence below. The time changing behaviour is given as `value` attribute of `Input`s.

In [5]:
with model:    
    rpm = tau/60
    # Time steps at first and corresponding speeds at second column
    rot_freq = Input(
        [[0, 3000*rpm],  
         [20, 6000*rpm]]  
    )  
      # Convection
    FilmLink(
        disc.surf.circ_faces,
        environment,
        film=forced_conv.rot_disc_in_air(
            rot_freq=rot_freq.value,  # Given as time dependent value
            rad=cyl.outer_rad,
        )
    )

The generated cutting heat increases after 20 seconds. This is also taken into account with an `Input` element. The heat loss itself can be modeled with a `HeatSource` element. It can take a function to describe time changing heat generation. The function is in this case created in place. It must take a parameter that contains the momentary element temperature at simulation time. This time the heat loss takes no temperature dependence in to account and is therefore ignored.

In [6]:
with model:
    # Columns for time steps and heat
    cutting_heat = Input(
        [[0, 40],   
         [20, 80]]  
    )
    HeatSource(
        disc.surf.outer,
        heat=lambda temp: cutting_heat.value
    )    

Non continuous input steps with rapid input changes should be considered by the simulation. Currently the solver does not notice input changes.

In [7]:
net = Network(model)
sim_span = 50.
result = net.sim([0., sim_span])

Simulation run progress, speed as sim. time vs. real time:
 0% 100% done.


As the result inner and outer disc temperatures are extracted form the result data.

In [8]:
result_times = linspace(0, sim_span, 11)
set_printoptions(formatter={'float': '{: 0.1f}'.format})
print("times            : ", result_times)
print("Outer disc temps.: ", result[disc.surf.outer].temp(result_times))
print("Inner disc temps.: ", result[disc.surf.inner].temp(result_times))

times            :  [ 0.0  5.0  10.0  15.0  20.0  25.0  30.0  35.0  40.0  45.0  50.0]
Outer disc temps.:  [ 20.0  36.4  44.9  51.6  59.3  79.9  93.6  105.0  115.0  124.0  132.2]
Inner disc temps.:  [ 20.0  20.5  20.9  21.4  21.9  24.3  26.6  28.8  31.1  33.3  35.6]
